# Flight Ticket Price

In [74]:
# Importing Packages from Python 

#Importing the packages pandas,Numpy,SeaBorn,matplotlib,math and Numpy 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy import stats
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag

%matplotlib inline

In [75]:
train_data=pd.read_excel('C:/Users/Suraj/Jupyter Projects/Project 12_FlightTicketPrice/Data_Train.xlsx')

train_data.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [76]:
test_data = pd.read_excel('C:/Users/Suraj/Jupyter Projects/Project 12_FlightTicketPrice/Test_set.xlsx')

test_data.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [77]:
big_data=pd.concat([train_data,test_data])
print(big_data.shape)

(13354, 11)


C:\Users\Suraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [78]:
big_data['Date']=big_data['Date_of_Journey'].str.split('/').str[0]
big_data['Month']=big_data['Date_of_Journey'].str.split('/').str[1]
big_data['Year']=big_data['Date_of_Journey'].str.split('/').str[2]

In [79]:
# Extracting details from Arrival Time

big_data['Arrival_Time'] = big_data['Arrival_Time'].str.split(' ').str[0]
big_data['Arrival_Hour'] = big_data['Arrival_Time'].str.split(':').str[0].astype(int)
big_data['Arrival_Min'] = big_data['Arrival_Time'].str.split(':').str[1].astype(int)

In [80]:
big_data['Total_Stops'].fillna('0 stop',inplace=True)
big_data['Total_Stops'].isnull().sum()

0

In [81]:
# Extracting details of Stops

big_data['Total_Stops'].replace('non-stop','0 stop',inplace=True)
big_data['Stop'] = big_data['Total_Stops'].str.split(' ').str[0]
big_data['Stop']= big_data['Stop'].astype(int)

In [82]:
# Extracting details from Departure Time 

big_data['Dep_Hour'] = big_data['Dep_Time'].str.split(':').str[0].astype(int)
big_data['Dep_Min']=  big_data['Dep_Time'].str.split(':').str[1].astype(int)

In [83]:
# Formatting Route details 

big_data['Route'] = big_data['Route'].fillna("None")
big_data['Route_1'] = big_data['Route'].str.split('→ ').str[0]
big_data['Route_2'] = big_data['Route'].str.split('→ ').str[1]
big_data['Route_3'] = big_data['Route'].str.split('→ ').str[2]
big_data['Route_4'] = big_data['Route'].str.split('→ ').str[3]
big_data['Route_5'] = big_data['Route'].str.split('→ ').str[4]

In [84]:
# Filling all missing values with 'None'

big_data['Route_1'].fillna("None",inplace=True)
big_data['Route_2'].fillna("None",inplace=True)
big_data['Route_3'].fillna("None",inplace=True)
big_data['Route_4'].fillna("None",inplace=True)
big_data['Route_5'].fillna("None",inplace=True)

In [85]:
# Formatting Duration Hours 

big_data['Duration_Hr']= big_data['Duration'].str.split(' ').str[0]
big_data['Duration_Hr']= big_data['Duration_Hr'].str.strip('h')
big_data['Duration_Hr']= big_data['Duration_Hr'].apply(lambda x: '0' if 'm' in x else x)
big_data[big_data['Duration_Hr']=='*m']

,Additional_Info,Airline,Arrival_Time,Date_of_Journey,Dep_Time,Destination,Duration,Price,Route,Source,...,Arrival_Min,Stop,Dep_Hour,Dep_Min,Route_1,Route_2,Route_3,Route_4,Route_5,Duration_Hr


In [86]:
big_data['Duration_Min']= big_data['Duration'].str.split(' ').str[1]
big_data['Duration_Min']= big_data['Duration_Min'].str.strip('m')

In [87]:
# Checking for missing values
big_data['Duration_Min'].isnull().sum()

1286

In [88]:
# Filling all missing values with 0's 

big_data['Duration_Min'].fillna(0,inplace=True)
big_data['Duration_Min'].isnull().sum()

0

In [89]:
# Converting time variables to integer 

big_data['Duration_Hr']= big_data['Duration_Hr'].astype(int)
big_data['Duration_Min']= big_data['Duration_Min'].astype(int)

In [90]:
big_data.head()

,Additional_Info,Airline,Arrival_Time,Date_of_Journey,Dep_Time,Destination,Duration,Price,Route,Source,...,Stop,Dep_Hour,Dep_Min,Route_1,Route_2,Route_3,Route_4,Route_5,Duration_Hr,Duration_Min
0,No info,IndiGo,01:10,24/03/2019,22:20,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,...,0,22,20,BLR,DEL,None,None,None,2,50
1,No info,Air India,13:15,1/05/2019,05:50,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,...,2,5,50,CCU,IXR,BBI,BLR,None,7,25
2,No info,Jet Airways,04:25,9/06/2019,09:25,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,...,2,9,25,DEL,LKO,BOM,COK,None,19,0
3,No info,IndiGo,23:30,12/05/2019,18:05,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,...,1,18,5,CCU,NAG,BLR,None,None,5,25
4,No info,IndiGo,21:35,01/03/2019,16:50,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,...,1,16,50,BLR,NAG,DEL,None,None,4,45


In [91]:
# Converting Date,Month and Year to integer

big_data['Date']=big_data['Date'].astype(int)
big_data['Month']=big_data['Month'].astype(int)
big_data['Year']= big_data['Year'].astype(int)

In [92]:
# Applying Label Encoder to tranform variables 

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
big_data['Additional_Info']= le.fit_transform(big_data['Additional_Info'])
big_data['Airline']= le.fit_transform(big_data['Airline'])
big_data['Destination']=le.fit_transform(big_data['Destination'])
big_data['Source']= le.fit_transform(big_data['Source'])
big_data['Route_1']=le.fit_transform(big_data['Route_1'])
big_data['Route_2']=le.fit_transform(big_data['Route_2'])
big_data['Route_3']=le.fit_transform(big_data['Route_3'])
big_data['Route_4']=le.fit_transform(big_data['Route_4'])
big_data['Route_5']=le.fit_transform(big_data['Route_5'])

In [93]:
train= big_data[0:10683]
test= big_data[10683:]
test = test.drop('Price',axis=1)

In [94]:
# Segregating Data to X and y 

X = train.drop(['Arrival_Time','Total_Stops','Date_of_Journey','Duration','Dep_Time','Price','Route'],axis=1)
y = train['Price']

In [98]:
# Applying scaling to the variables

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [99]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [97]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [100]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [102]:
# Determining the intercept

print(lr.intercept_)

9083.151681125239


In [103]:
# Retrieving the coefficients

print(lr.coef_)

[ 2.93061246e+02  3.69131756e+02  5.05830131e+02 -1.93902971e+02
 -6.30911452e+02 -3.72247852e+02  5.68434189e-14 -1.06002303e+02
  9.10513145e+00  2.24217371e+03  1.37111825e+02 -3.34868670e+01
 -4.67047018e+02 -7.71399078e+02 -1.33426833e+03  5.84317841e+01
  1.34657636e+02  2.75045089e+02 -2.27002837e+02]


In [109]:
pred=lr.predict(X_test)
pred=pd.DataFrame(pred,columns=['Predicted'])
pred

,Predicted
0,9961.449311
1,6013.064410
2,8876.066098
3,10625.400865
4,11167.549549
5,5244.025418
6,8984.986868
7,6241.003510
8,8101.394688
9,10746.247418


In [110]:
# Evaluating the Algorithm

from sklearn import metrics
print('Mean Absolute Error:',metrics.mean_absolute_error(y_test,pred))
print('Mean Squared Error:',metrics.mean_squared_error(y_test,pred))
print('Root Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_test,pred)))


Mean Absolute Error: 2275.172899678753
Mean Squared Error: 10832748.19717235
Root Mean Squared Error: 3291.314053257809
